## Dependencies

In [57]:
#Import dependencies

#NFL pbp data
import nfl_data_py as nfl

#Basics / visualizations
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

#Machine learning tools
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#Stats Stuff
from scipy import stats
from scipy.stats import t

#Turn off max columns for pandas DataFrame
pd.set_option('display.max_columns', None)

#Turn off when showing off, switch comments to change
pd.options.mode.chained_assignment = None  # Disable the warning
# pd.options.mode.chained_assignment = 'warn'

## Importing and Prepping Schedules

In [2]:
schedules_df = nfl.import_schedules([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [3]:
def calculate_implied_totals(df):
    """
    Calculate the implied home and away team totals based on the spread and total lines.
    """
    #Implied totals based on the total and spread lines
    df['implied_home_total'] = (df['total_line'] + df['spread_line']) / 2
    df['implied_away_total'] = (df['total_line'] - df['spread_line']) / 2
    
    return df

In [4]:
schedules_df = calculate_implied_totals(schedules_df)

In [5]:
schedules_df.drop(columns= ['away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium', 'away_score', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game', 'away_qb_id', 'home_qb_id', 'gameday', 'weekday',
       'gametime'], inplace=True)

In [6]:
#Create a DataFrame for away quarterbacks
away_df = schedules_df[['game_id', 'season', 'game_type', 'week', 'away_team', 'home_team', 'away_qb_name', 'implied_away_total', 'roof', 'surface',]].copy()
away_df.rename(columns={'away_qb_name': 'qb_name', 'implied_away_total': 'implied_total'}, inplace=True)
away_df['home_away'] = 'away'
away_df['posteam'] = away_df['away_team']
away_df['defteam'] = away_df['home_team']

#Create a DataFrame for home quarterbacks
home_df = schedules_df[['game_id', 'season', 'game_type', 'week', 'away_team', 'home_team', 'home_qb_name', 'implied_home_total', 'roof', 'surface']].copy()
home_df.rename(columns={'home_qb_name': 'qb_name', 'implied_home_total': 'implied_total'}, inplace=True)
home_df['home_away'] = 'home'
home_df['posteam'] = home_df['home_team']
home_df['defteam'] = home_df['away_team']

#Concatenate both DataFrames
combined_schedule_df = pd.concat([away_df, home_df])

#Reset index for cleanliness
combined_schedule_df.reset_index(drop=True, inplace=True)

In [7]:
def format_passer_name(qb_name):
    if pd.isna(qb_name):  #Check if the name is NaN
        return ""
    
    name_parts = qb_name.split()
    
    #Extract the first name and last name
    first_name = name_parts[0]
    last_name = name_parts[-1]  # Last name should always be the last part
    
    return f"{first_name[0]}.{last_name}"

#Apply the function to create the new 'player_passer_name' column
combined_schedule_df['passer_player_name'] = combined_schedule_df['qb_name'].apply(format_passer_name)

In [8]:
#Reorder Columns
combined_schedule_df = combined_schedule_df[['game_id', 'season', 'game_type', 'week', 'roof', 'surface',
                                             'posteam', 'defteam', 'home_team', 'away_team', 'qb_name', 'passer_player_name', 'implied_total', 'home_away']]

## Importing PBP Training Data

In [9]:
#Select only the relevant columns
columns = ['game_id', 'passer_player_name', 'posteam', 'defteam', 'season', 'week', 'home_team', 'away_team', 'play_type', 'air_yards', 
           'yards_after_catch', 'epa', 'complete_pass', 'incomplete_pass', 'interception', 'qb_hit', 'sack', 'pass_touchdown',
           'passing_yards', 'cpoe', 'roof', 'surface']

#Loading in the NFL pbp data
data = nfl.import_pbp_data(range(2014,2024 + 1), columns, include_participation=False)

#nfl-data-py still loads other columns, so we again need to set our data equal to only the columns we want
data = data[columns]

#Drop all rows that are not a pass
data = data[data['play_type'] == 'pass']

#Drop the play type column
passer_data = data.drop(columns=['play_type'])

2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


## Passer DF Training Prep

In [10]:
#Group the data together by passer, week, season and aggregate
passer_df = passer_data.groupby(['game_id', 'passer_player_name', 'week', 'season'], as_index=False).agg(
    {'posteam' : 'first',
     'defteam' : 'first',
     'home_team' : 'first',
     'away_team' : 'first',
     'air_yards' : 'sum',
     'yards_after_catch' : 'sum',
     'epa' : 'sum',
     'complete_pass' : 'sum',
     'incomplete_pass' : 'sum',
     'interception' : 'sum',
     'qb_hit' : 'sum',
     'sack' : 'sum',
     'pass_touchdown' : 'sum',
     'passing_yards' : 'sum',
     'cpoe' : 'mean',
     'roof' : 'first',
     'surface' : 'first'
     }
)

#Create a new column that is completion percentage
passer_df['completion_percentage'] = passer_df['complete_pass'] / (passer_df['complete_pass'] + passer_df['incomplete_pass'])

#Create a new column that is the number of pass attempts
passer_df['pass_attempts'] = passer_df['complete_pass'] + passer_df['incomplete_pass']

#Create a new column that equals 1 if the passer is the home team and 0 if the passer is the away team
passer_df['home_flag'] = passer_df['home_team'] == passer_df['posteam']

#Reorder the columns
passer_df = passer_df[['home_team', 'away_team', 'complete_pass', 'incomplete_pass' ,'game_id', 'passer_player_name', 'posteam', 'defteam', 'season', 'week', 'passing_yards', 'home_flag', 'completion_percentage', 'pass_attempts',
                       'air_yards',  'yards_after_catch', 'epa', 'interception', 'qb_hit', 'sack', 'pass_touchdown', 
                        'cpoe', 'roof', 'surface']]

## Defense DF Training Prep

In [11]:
#Select only the relevant columns
defense_columns = ['defteam', 'season', 'week', 'home_team', 'away_team', 'play_type', 'air_yards',
                   'yards_after_catch', 'epa', 'complete_pass', 'incomplete_pass', 'interception', 'qb_hit', 'sack', 'pass_touchdown',
                   'passing_yards', 'cpoe', 'roof', 'surface']


#nfl-data-py still loads other columns, so we again need to set our data equal to only the columns we want
defense_data = data[defense_columns]

#Drop the play type column
defense_data = defense_data.drop(columns=['play_type'])

#Group the data together by passer, week, season and aggregate
defense_df = defense_data.groupby(['defteam', 'week', 'season'], as_index=False).agg(
    {'home_team': 'first',
     'away_team': 'first',
     'air_yards': 'sum',
     'yards_after_catch': 'sum',
     'epa': 'sum',
     'complete_pass': 'sum',
     'incomplete_pass': 'sum',
     'interception': 'sum',
     'qb_hit': 'sum',
     'sack': 'sum',
     'pass_touchdown': 'sum',
     'passing_yards': 'sum',
     'cpoe': 'mean',
     'roof': 'first',
     'surface': 'first'
     }
)

#Create a new column that is completion percentage
defense_df['completion_percentage'] = defense_df['complete_pass'] / (defense_df['complete_pass'] + defense_df['incomplete_pass'])

#Create a new column that is the number of pass attempts
defense_df['pass_attempts'] = defense_df['complete_pass'] + defense_df['incomplete_pass']

#Create a new column that equals 1 if the defense is the home team and 0 if the defense is the away team
defense_df['home_flag'] = defense_df['home_team'] == defense_df['defteam']

#Reorder the columns
defense_df = defense_df[['home_team', 'away_team', 'complete_pass', 'incomplete_pass', 'defteam', 'season', 'week', 'home_flag', 'passing_yards', 'completion_percentage', 'pass_attempts',
                       'air_yards',  'yards_after_catch', 'epa', 'interception', 'qb_hit', 'sack', 'pass_touchdown', 
                       'cpoe', 'roof', 'surface']]

## Preprocessing, namely exponentially weighted moving averages for offensive and defensive stats to be used as features in train set

In [12]:
def calculate_offensive_ewma(passer_df):
    """
    Calculates EWMA for offensive columns using previous weeks' data, ensuring no leakage by excluding the current week.
    Takes into account multiple seasons.
    """
    #Sort by passer, season, and week
    passer_df = passer_df.sort_values(by=['passer_player_name', 'season', 'week'])

    #Calculate the exponentially weighted moving average for each offensive feature, excluding the current week
    passer_df['completion_percentage_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['completion_percentage']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['pass_attempts_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['pass_attempts']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['air_yards_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['air_yards']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['yards_after_catch_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['yards_after_catch']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['epa_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['epa']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['interception_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['interception']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['qb_hit_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['qb_hit']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['sack_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['sack']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['pass_touchdown_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['pass_touchdown']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['passing_yards_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['passing_yards']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    passer_df['cpoe_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['cpoe']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    return passer_df

In [13]:
def pass_cleaner(passer_df):
    """
    Preps passer df for merging; drops unnecessary columns
    """
    passer_df.drop(columns=['home_team', 'away_team', 'complete_pass', 'incomplete_pass', 'completion_percentage', 'air_yards', 'yards_after_catch', 'epa', 
                                    'interception', 'qb_hit', 'sack', 'pass_touchdown', 'cpoe', 'home_team', 'away_team', 
                                    'complete_pass', 'incomplete_pass'], inplace=True)
    
    return passer_df

In [14]:
passer_emwa = calculate_offensive_ewma(passer_df)

In [15]:
passer_prepped = pass_cleaner(passer_emwa)

In [16]:
def calculate_defensive_ewma(defense_df):
    """
    Calculates EWMA for defensive columns using previous weeks' data (excluding the current week).
    """
    #Sort by 'defteam', 'season', and 'week' in ascending order (to ensure time order)
    defense_df = defense_df.sort_values(by=['defteam', 'season', 'week'])

    #Ensure proper grouping by both defteam and season
    defense_df['completion_percentage_ewma'] = defense_df.groupby(['defteam', 'season'])['completion_percentage']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())
    
    defense_df['pass_attempts_ewma'] = defense_df.groupby(['defteam', 'season'])['pass_attempts']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['air_yards_ewma'] = defense_df.groupby(['defteam', 'season'])['air_yards']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['yards_after_catch_ewma'] = defense_df.groupby(['defteam', 'season'])['yards_after_catch']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['epa_ewma'] = defense_df.groupby(['defteam', 'season'])['epa']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['interception_ewma'] = defense_df.groupby(['defteam', 'season'])['interception']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['qb_hit_ewma'] = defense_df.groupby(['defteam', 'season'])['qb_hit']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['sack_ewma'] = defense_df.groupby(['defteam', 'season'])['sack']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['pass_touchdown_ewma'] = defense_df.groupby(['defteam', 'season'])['pass_touchdown']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['passing_yards_ewma'] = defense_df.groupby(['defteam', 'season'])['passing_yards']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    defense_df['cpoe_ewma'] = defense_df.groupby(['defteam', 'season'])['cpoe']\
        .transform(lambda x: x.shift().ewm(min_periods=1, span=5).mean())

    return defense_df

In [17]:
def def_cleaner(defense_df):
    """
    Preps passer df for merging; drops unnecessary columns
    """
    #Drop the non-ewma columns
    defense_df.drop(columns=['passing_yards','completion_percentage',
                            'air_yards', 'yards_after_catch', 'epa',     
                            'interception', 'qb_hit', 'sack', 'pass_touchdown', 'pass_attempts', 'cpoe', 'complete_pass', 'incomplete_pass',
                            'home_team', 'away_team'
                            ], inplace=True)
    
    return defense_df

In [18]:
defense_ewma = calculate_defensive_ewma(defense_df)

In [19]:
defense_prepped = def_cleaner(defense_ewma)

In [20]:
#Merge the defense and passer dataframes together
full_with_ewma = passer_prepped.merge(defense_prepped, how='inner', on=['defteam', 'season', 'week', 'roof', 'surface'], suffixes=('_passer', '_defense'))

#Get rid of flukey rows
filtered_with_ewma = full_with_ewma[(full_with_ewma['pass_attempts'] > 8) & (full_with_ewma['passing_yards'] >= 10)]

#Ford Field is empty, you hate to see that
filtered_with_ewma['surface'].replace('', 'fieldturf', inplace=True)

## Week 1 is NaN's (duh, we use past weeks for EWMA's) for MVP I'll just get rid but for future seasons/rookies I do need a strategy

In [21]:
filtered_with_ewma = filtered_with_ewma.dropna()

In [22]:
#Perform the merge based on game_id, season, week, and team
filtered_with_ewma_it = pd.merge(filtered_with_ewma, combined_schedule_df[['game_id', 'season', 'week', 'posteam', 'implied_total']], 
                     on=['game_id', 'season', 'week', 'posteam'], 
                     how='inner')

## Modeling

In [23]:
#Define the target and features
target = 'passing_yards'
features = filtered_with_ewma_it.columns.drop(['pass_attempts', 'passer_player_name', 'passing_yards', 'posteam', 'defteam', 'season', 'week', 'game_id', 'implied_total'])

#Define categorical and numeric columns
categorical_columns = ['roof', 'surface', 'home_flag_passer', 'home_flag_defense']
numeric_columns = [col for col in features if col not in categorical_columns]

#Split the data into training (up to 2023) and testing (2024)
train_data = filtered_with_ewma_it[filtered_with_ewma_it['season'] <= 2023]
test_data = filtered_with_ewma_it[filtered_with_ewma_it['season'] == 2024]

#Separate features and target for both training and testing
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

#Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns),
        ('num', StandardScaler(), numeric_columns)
    ])

#Apply the transformations to the training and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [24]:
#Initialize the basic XGBoost model
xgb_model_basic = xgb.XGBRegressor(
    n_estimators=142,  # Basic number of boosting rounds
    learning_rate=0.101,  # Standard learning rate
    max_depth=3,        # Typical depth for a basic model
    reg_lambda= 2,      #something something regularization
    random_state=42,    # For reproducibility
    n_jobs=-1           # Use all CPU cores for faster training
)

#Train the model on the entire pre-2024 data
xgb_model_basic.fit(X_train_transformed, y_train)

#Make predictions on the 2024 season (X_test)
y_pred = xgb_model_basic.predict(X_test_transformed)

#Calculate RMSE for the 2024 season predictions
rmse_basic = mean_squared_error(y_test, y_pred, squared=False)
print(f"Basic RMSE on the 2024 season: {rmse_basic}")

#Attach predictions back to the original test DataFrame
test_data['predicted_passing_yards'] = y_pred

#Remove the rows with the largest and smallest passing yards
max_passing_yards = test_data['passing_yards'].max()
min_passing_yards = test_data['passing_yards'].min()

#Filter the test_data to exclude the rows with max and min passing yards
filtered_test_data = test_data[(test_data['passing_yards'] != max_passing_yards) & (test_data['passing_yards'] != min_passing_yards)]

#Calculate RMSE for the remaining data
rmse_filtered = mean_squared_error(filtered_test_data['passing_yards'], filtered_test_data['predicted_passing_yards'], squared=False)
print(f"RMSE on the 2024 season without the min and max true passing yard games: {rmse_filtered}")

Basic RMSE on the 2024 season: 75.5245132446289
RMSE on the 2024 season without the min and max true passing yard games: 72.85186004638672


# ^ this error score sucks (bad!) best is around 65

In [25]:
#Ensure test_data is just the 2024 season
test_data = filtered_with_ewma_it[filtered_with_ewma_it['season'] == 2024]

In [26]:
#Attach predictions back to the original test DataFrame
test_data['predicted_passing_yards'] = y_pred

In [66]:
#Calculate whether the model over-predicted or under-predicted to visualize trends in predictions across weeks
test_data['pyoe'] = test_data['passing_yards'] - test_data['predicted_passing_yards']

#To be used in monte carlo simulator
global_residual_std = test_data['pyoe'].std()

In [29]:
average_negative = test_data[test_data['pyoe'] < 0]['pyoe'].mean()
average_positive = test_data[test_data['pyoe'] > 0]['pyoe'].mean()


(-62.63853, 58.73861)

## Current Week Preprocessing

In [30]:
def calculate_ewma_tester_off(passer_df):
    """
    Calculates EWMA for offensive columns using the current and previous weeks' data, including the current week.
    Takes into account multiple seasons.
    """
    #Sort by passer, season, and week
    passer_df = passer_df.sort_values(by=['passer_player_name', 'season', 'week'])

    #Calculate the exponentially weighted moving average for each offensive feature, including the current week
    passer_df['completion_percentage_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['completion_percentage']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['pass_attempts_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['pass_attempts']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['air_yards_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['air_yards']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['yards_after_catch_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['yards_after_catch']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['epa_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['epa']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['interception_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['interception']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['qb_hit_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['qb_hit']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['sack_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['sack']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['pass_touchdown_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['pass_touchdown']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['passing_yards_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['passing_yards']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    passer_df['cpoe_ewma'] = passer_df.groupby(['passer_player_name', 'season'])['cpoe']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    return passer_df

In [31]:
passer_df_tester = passer_df[(passer_df['season'] == 2024)]

In [32]:
prediction_set_passer_nc = calculate_ewma_tester_off(passer_df_tester)

In [33]:
prediction_set_passer = pass_cleaner(prediction_set_passer_nc)

In [34]:
#Get rid of flukey rows and get current week offensive stuff ready to join on a prepped schedules DF
prediction_set_passer_recents = prediction_set_passer[(prediction_set_passer['pass_attempts'] > 8) & (prediction_set_passer['passing_yards'] >= 10)]

prediction_set_passer_recents = prediction_set_passer_recents.loc[prediction_set_passer_recents.groupby('posteam')['week'].idxmax()]

prediction_set_passer_recents['surface'].replace('', 'fieldturf', inplace=True)

prediction_set_passer_recents.drop(columns=['game_id', 'passer_player_name', 'defteam', 'roof', 'surface', 'home_flag', 'passing_yards', 'pass_attempts'], inplace=True)

In [35]:
def calculate_ewma_tester_def(defense_df):
    """
    Calculates EWMA for defensive columns using previous weeks' data (excluding the current week).
    """
    #Sort by 'defteam', 'season', and 'week' in ascending order (to ensure time order)
    defense_df = defense_df.sort_values(by=['defteam', 'season', 'week'])

    #Ensure proper grouping by both defteam and season
    defense_df['completion_percentage_ewma'] = defense_df.groupby(['defteam', 'season'])['completion_percentage']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())
    
    defense_df['pass_attempts_ewma'] = defense_df.groupby(['defteam', 'season'])['pass_attempts']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['air_yards_ewma'] = defense_df.groupby(['defteam', 'season'])['air_yards']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['yards_after_catch_ewma'] = defense_df.groupby(['defteam', 'season'])['yards_after_catch']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['epa_ewma'] = defense_df.groupby(['defteam', 'season'])['epa']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['interception_ewma'] = defense_df.groupby(['defteam', 'season'])['interception']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['qb_hit_ewma'] = defense_df.groupby(['defteam', 'season'])['qb_hit']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['sack_ewma'] = defense_df.groupby(['defteam', 'season'])['sack']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['pass_touchdown_ewma'] = defense_df.groupby(['defteam', 'season'])['pass_touchdown']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['passing_yards_ewma'] = defense_df.groupby(['defteam', 'season'])['passing_yards']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    defense_df['cpoe_ewma'] = defense_df.groupby(['defteam', 'season'])['cpoe']\
        .transform(lambda x: x.ewm(min_periods=1, span=5).mean())

    return defense_df

In [36]:
defense_df_tester = defense_df[defense_df['season'] == 2024]

In [37]:
prediction_set_defense_nc = calculate_ewma_tester_def(defense_df_tester)

In [38]:
prediction_set_defense = def_cleaner(prediction_set_defense_nc)

In [39]:
prediction_set_defense_recents = prediction_set_defense.loc[prediction_set_defense.groupby('defteam')['week'].idxmax()]

prediction_set_defense_recents['surface'].replace('', 'fieldturf', inplace=True)

prediction_set_defense_recents.drop(columns=['roof', 'surface', 'home_flag'], inplace=True)

In [64]:
def update_week_to_next(df):
    """
    Function to update all teams to the next NFL week based on the current max week.
    Teams that are behind the max week will jump to the next NFL week.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing NFL data including the 'week' column.
    
    Returns:
    pd.DataFrame: DataFrame with 'week' values updated appropriately.
    """
    df_copy = df.copy()

    #Current week
    current_week = df_copy['week'].max()

    #Increment all weeks to the next NFL week (based on current max week)
    df_copy['week'] = df_copy['week'].apply(lambda x: current_week + 1)
    
    return df_copy

In [41]:
current_week_offense_set = update_week_to_next(prediction_set_passer_recents)
current_week_defense_set = update_week_to_next(prediction_set_defense_recents)

In [42]:
current_week_defense_set.rename(columns={
    'completion_percentage_ewma': 'completion_percentage_ewma_defense',
    'pass_attempts_ewma': 'pass_attempts_ewma_defense',
    'air_yards_ewma': 'air_yards_ewma_defense',
    'yards_after_catch_ewma': 'yards_after_catch_ewma_defense',
    'epa_ewma': 'epa_ewma_defense',
    'interception_ewma': 'interception_ewma_defense',
    'qb_hit_ewma': 'qb_hit_ewma_defense',
    'sack_ewma': 'sack_ewma_defense',
    'pass_touchdown_ewma': 'pass_touchdown_ewma_defense',
    'passing_yards_ewma': 'passing_yards_ewma_defense',
    'cpoe_ewma': 'cpoe_ewma_defense'
}, inplace=True)

In [43]:
current_week_offense_set.rename(columns={
    'completion_percentage_ewma': 'completion_percentage_ewma_passer',
    'pass_attempts_ewma': 'pass_attempts_ewma_passer',
    'air_yards_ewma': 'air_yards_ewma_passer',
    'yards_after_catch_ewma': 'yards_after_catch_ewma_passer',
    'epa_ewma': 'epa_ewma_passer',
    'interception_ewma': 'interception_ewma_passer',
    'qb_hit_ewma': 'qb_hit_ewma_passer',
    'sack_ewma': 'sack_ewma_passer',
    'pass_touchdown_ewma': 'pass_touchdown_ewma_passer',
    'passing_yards_ewma': 'passing_yards_ewma_passer',
    'cpoe_ewma': 'cpoe_ewma_passer'
}, inplace=True)

In [44]:
combined_schedule_preds = combined_schedule_df[combined_schedule_df['season'] == combined_schedule_df['season'].max()]
combined_schedule_preds.drop(columns=['home_away'], inplace=True)

In [45]:
#First merge stitching features
off_plus_schedule = pd.merge(
    current_week_offense_set, 
    combined_schedule_preds, 
    on=['season', 'week', 'posteam'], 
    suffixes=('_fromoff', '_fromschedule'), 
    how='inner'
)

In [46]:
#Here comes the defensive stuff
full_pred_features = pd.merge(
    off_plus_schedule,
    current_week_defense_set,
    on=['season', 'week', 'defteam'],  #Merging on these columns
    suffixes=('_fromscheduleAHHHHHH', '_DEFENSEHUGSYDEFENSEHUGSY'), #this should make it obvious if there's an error
    how='inner'
)


In [47]:
#List of the new column order
new_column_order = ['game_id', 'season', 'week','game_type','posteam', 'defteam', 'home_team', 'away_team', 'qb_name', 'passer_player_name', 
              'implied_total', 'roof', 'surface',
              'completion_percentage_ewma_passer', 'pass_attempts_ewma_passer',
              'air_yards_ewma_passer', 'yards_after_catch_ewma_passer',
              'epa_ewma_passer', 'interception_ewma_passer', 'qb_hit_ewma_passer',
              'sack_ewma_passer', 'pass_touchdown_ewma_passer',
              'passing_yards_ewma_passer', 'cpoe_ewma_passer', 'completion_percentage_ewma_defense',
              'pass_attempts_ewma_defense', 'air_yards_ewma_defense',
              'yards_after_catch_ewma_defense', 'epa_ewma_defense',
              'interception_ewma_defense', 'qb_hit_ewma_defense', 'sack_ewma_defense',
              'pass_touchdown_ewma_defense', 'passing_yards_ewma_defense',
              'cpoe_ewma_defense']

#Reorder inplace
full_pred_features = full_pred_features[new_column_order]

#Create the home_flag_passer column
full_pred_features['home_flag_passer'] = full_pred_features['posteam'] == full_pred_features['home_team']

#Create the home_flag_defense column
full_pred_features['home_flag_defense'] = full_pred_features['defteam'] == full_pred_features['home_team']

In [48]:
def predict_passing_yards(df, model, preprocessor, features, categorical_columns, numeric_columns, target_column='predicted_passing_yards'):
    """
    Function to make passing yard predictions on a new dataframe.
    
    Parameters:
    new_data (pd.DataFrame): The new dataframe containing feature columns.
    model (xgb.XGBRegressor): Trained XGBoost model.
    preprocessor (ColumnTransformer): Preprocessor object for transforming the features.
    features (list): List of feature columns used by the model.
    categorical_columns (list): List of categorical columns to be preprocessed.
    numeric_columns (list): List of numeric columns to be preprocessed.
    target_column (str): The name of the column for storing the predictions. Default is 'predicted_passing_yards'.
    
    Returns:
    pd.DataFrame: The input dataframe with an added column of predicted passing yards.
    """
    
    #Select and transform the features from the new data
    X_new = df[features]
    
    #Apply preprocessing (scaling and encoding)
    X_new_transformed = preprocessor.transform(X_new)
    
    #Make predictions using the trained model
    df[target_column] = model.predict(X_new_transformed)
    
    return df

#Example usage
upcoming_week_predictions = predict_passing_yards(full_pred_features, xgb_model_basic, preprocessor, features, categorical_columns, numeric_columns)

In [95]:
#get o/u's

upcoming_week_predictions['pass_yards_ou'] = upcoming_week_predictions['predicted_passing_yards'].apply(lambda x: round(x * 2) / 2)

clean_predictions = upcoming_week_predictions[['game_id', 'season', 'week', 'game_type', 'posteam', 'defteam',
       'home_team', 'away_team', 'qb_name', 'predicted_passing_yards', 'pass_yards_ou',
       'implied_total']]

In [96]:
# Group by 'posteam' and aggregate 'pyoe' values into a list
pyoe_list = test_data.groupby('posteam')['pyoe'].apply(list).reset_index()

# Rename the 'pyoe' column to 'pyoe_list' to reflect that it now contains lists of values
pyoe_list.rename(columns={'pyoe': 'pyoe_list'}, inplace=True)

# Sort by the length of the list or by other criteria if needed (e.g., by the number of games played)
pyoe_list.sort_values(by='posteam', inplace=True)


In [97]:
clean_predictions = pd.merge(pyoe_list, clean_predictions, on='posteam', how='left')

In [162]:
clean_predictions = clean_predictions[['game_id', 'season', 'week', 'game_type', 'implied_total', 'posteam', 'defteam',
       'home_team', 'away_team', 'qb_name', 'predicted_passing_yards', 'pass_yards_ou', 'pyoe_list']]

In [145]:
from scipy.stats import truncnorm

def monte_carlo_simulations_truncated(df, global_residual_std, residual_col='pyoe_list',
                                      predicted_col='predicted_passing_yards', num_simulations=1000,
                                      noise_scale=0.25, min_yards=50, max_yards=650):
    """
    Monte Carlo simulation for all QBs using truncated normal distribution to ensure realistic passing yard values.
    :param df: DataFrame containing QBs with predicted means and mean team residuals
    :param global_residual_std: Global residual standard deviation to use for the distribution
    :param residual_col: Column containing team-specific mean residuals
    :param predicted_col: Column containing predicted passing yards for each QB
    :param num_simulations: Number of Monte Carlo simulations to run per QB
    :param noise_scale: Standard deviation of the added noise
    :param min_yards: Minimum passing yards to ensure no unrealistic low values
    :param max_yards: Maximum passing yards to cap extreme outliers
    :return: Dictionary of QBs with their simulated passing yard distributions
    """
    qb_simulations = {}

    for idx, row in df.iterrows():
        qb_name = row['qb_name']
        predicted_mean = row[predicted_col]
        mean_team_residual = row[residual_col]

        # Calculate the truncated normal distribution parameters
        lower_bound = (min_yards - predicted_mean) / global_residual_std
        upper_bound = (max_yards - predicted_mean) / global_residual_std

        # Sample from the truncated normal distribution
        truncated_simulations = truncnorm.rvs(lower_bound, upper_bound, loc=predicted_mean, scale=global_residual_std, size=num_simulations)

        # Add a small amount of noise to introduce variability
        noise = np.random.normal(loc=0, scale=noise_scale, size=num_simulations)
        simulations = truncated_simulations + noise

        # Ensure no negative values by applying the minimum passing yard threshold
        simulations = np.maximum(min_yards, simulations)

        # Save the simulations for this QB
        qb_simulations[qb_name] = simulations

    return qb_simulations

In [146]:
qb_simulations = monte_carlo_simulations_truncated(clean_predictions, global_residual_std)

In [169]:
def implied_odds_to_american(implied_prob):
    """
    Convert implied probability to American odds format.
    :param implied_prob: Implied probability (between 0 and 1)
    :return: American odds (with + for positive odds)
    """
    if implied_prob == 0:
        return 'N/A'
    if implied_prob >= 0.5:
        american_odds = -100 / implied_prob
    else:
        american_odds = (1 / implied_prob - 1) * 100
    
    # If positive odds, add the "+" sign in front
    if american_odds > 0:
        return f"+{round(american_odds)}"
    else:
        return round(american_odds)

In [170]:
def midpoint_madness(qb_name, qb_simulations, over_qbs=[], under_qbs=[]):
    """
    Function to simulate over/under passing yard predictions in Midpoint Madness game.
    
    :param qb_name: The name of the main QB to start with.
    :param qb_simulations: Dictionary containing simulated passing yard results for all QBs.
    :param over_qbs: List of QB names that the user thinks will pass for fewer yards than the main QB.
    :param under_qbs: List of QB names that the user thinks will pass for more yards than the main QB.
    
    :return: Dictionary of probabilities or betting odds for each over/under comparison.
    """
    results = {}
    combined_prob = 1  # To calculate the combined probability

    # Simulations for the chosen QB
    main_qb_simulations = qb_simulations[qb_name]
    
    # Calculate probabilities for 'over' comparisons
    for over_qb in over_qbs:
        over_qb_simulations = qb_simulations[over_qb]
        prob_main_qb_more = np.mean(main_qb_simulations > over_qb_simulations)
        results[f"{qb_name} > {over_qb}"] = {
            'probability': prob_main_qb_more,
            'american_odds': implied_odds_to_american(prob_main_qb_more)
        }
        # Update the combined probability
        combined_prob *= prob_main_qb_more
    
    # Calculate probabilities for 'under' comparisons
    for under_qb in under_qbs:
        under_qb_simulations = qb_simulations[under_qb]
        prob_main_qb_less = np.mean(main_qb_simulations < under_qb_simulations)
        results[f"{qb_name} < {under_qb}"] = {
            'probability': prob_main_qb_less,
            'american_odds': implied_odds_to_american(prob_main_qb_less)
        }
        # Update the combined probability
        combined_prob *= prob_main_qb_less
    
    # Add combined probability to the results
    results['combined_probability'] = combined_prob
    results['combined_american_odds'] = implied_odds_to_american(combined_prob)
    
    return results

In [175]:
# Run the Midpoint Madness function for Kyler Murray
results = midpoint_madness(
    qb_name='Kyler Murray',
    qb_simulations=qb_simulations,
    over_qbs=['Bryce Young', 'Lamar Jackson'],  # QBs you think Kyler will pass more than
    under_qbs=['Josh Allen', 'Jared Goff']  # QBs you think Kyler will pass fewer than
)

# Display results
for comparison, data in results.items():
    if comparison == 'combined_probability':
        # For combined probability (which is a float), format as a number
        print(f"{comparison}: {data:.2f}")
    elif comparison == 'combined_american_odds':
        # Print combined American odds directly since it's a string
        print(f"{comparison}: {data}")
    else:
        # For comparison results, format probability and print American odds as a string
        print(f"{comparison}: Probability = {data['probability']:.2f}, American Odds = {data['american_odds']}")

Kyler Murray > Bryce Young: Probability = 0.63, American Odds = -160
Kyler Murray > Lamar Jackson: Probability = 0.32, American Odds = +214
Kyler Murray < Josh Allen: Probability = 0.64, American Odds = -155
Kyler Murray < Jared Goff: Probability = 0.73, American Odds = -137
combined_probability: 0.09
combined_american_odds: +969


## Charts for model stuff if I need to look. It's a pretty shit model. It does ok I guess but not a lot of variance in projection lmao

In [168]:
#sanity checker

# min_max_results = {}
# for qb_name, simulations in qb_simulations.items():
#     min_max_results[qb_name] = {
#         'min_passing_yards': np.min(simulations),
#         'max_passing_yards': np.max(simulations)
#     }

# # Convert the min and max passing yards results into a DataFrame
# min_max_df = pd.DataFrame.from_dict(min_max_results, orient='index').reset_index()
# min_max_df.rename(columns={'index': 'qb_name'}, inplace=True)

# min_max_df.sort_values(by='min_passing_yards')

In [157]:
# # Plotting the histogram of PYOE
# plt.figure(figsize=(10, 6))
# plt.hist(test_data['pyoe'], bins=25, edgecolor='black', alpha=0.7)
# plt.title('Distribution of Pass Yards Over Expected (PYOE)', fontsize=16)
# plt.xlabel('PYOE (Predicted Passing Yards - Actual Passing Yards)', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.grid(True, alpha=0.5)

# # Show the plot
# plt.show()

In [158]:
# # Plotting the histogram of PYOE
# plt.figure(figsize=(10, 6))
# plt.hist(test_data['predicted_passing_yards'], bins=30, edgecolor='black', alpha=0.7)
# plt.title('Distribution of Predicted Pass Yards', fontsize=16)
# plt.xlabel('Passing Yards', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.grid(True, alpha=0.5)

# # Show the plot
# plt.show()

In [159]:
# # Plotting the histogram of PYOE
# plt.figure(figsize=(10, 6))
# plt.hist(test_data['passing_yards'], bins=30, edgecolor='black', alpha=0.7)
# plt.title('Distribution of Pass Yards', fontsize=16)
# plt.xlabel('Passing Yards', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.grid(True, alpha=0.5)

# # Show the plot
# plt.show()

In [160]:
# # Plotting the histogram of predicted passing yards and true passing yards
# plt.figure(figsize=(10, 6))

# # Plot histogram for true passing yards
# plt.hist(test_data['passing_yards'], bins=30, alpha=0.5, label='True Passing Yards', color='blue')

# # Plot histogram for predicted passing yards
# plt.hist(test_data['predicted_passing_yards'], bins=30, alpha=0.5, label='Predicted Passing Yards', color='orange')

# # Adding labels and title
# plt.xlabel('Passing Yards')
# plt.ylabel('Frequency')
# plt.title('Histogram of True vs Predicted Passing Yards on 2024 Season, Bins = 25')
# plt.legend(loc='upper right')

# # Show the plot
# plt.show()

below checks a qb's simulator prediction with a bootstrapped curve that is custom to that team's own passing variance this year. rad

In [ ]:
# # Example: Visualize Kyler Murray's simulated passing yards
# kyler_murray_simulations = qb_simulations['Daniel Jones']

# # Example: Visualize Kyler Murray's simulated passing yards using KDE plot
# plt.figure(figsize=(10, 6))
# sns.kdeplot(kyler_murray_simulations, fill=True)

# # Add titles and labels
# plt.title("KDE of Simulated Passing Yards for Kyler Murray", fontsize=14)
# plt.xlabel("Passing Yards", fontsize=12)
# plt.ylabel("Density", fontsize=12)

# # Show the plot

# plt.show()

## To do:
* Figure out wtm is when there is a new qb/injury
* Find a way to tone down overpredictions (seems to be a weird trend this season people passing for less yards)
* Week 1 stuff for next season
* Write predictions to some sort of database for front end use? (ask pals with better knowledge on usable coding skills)